In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

# load local model file and initialize model class
from model import ModelClass
model = ModelClass() # set a few number of periods.


par = model.par
sol = model.sol
sim = model.sim

f:\MPC\Studenter\Speciale\Peter og Mathias\Code Peter\Design_retirement\model.py:69: SyntaxWarning: invalid escape sequence '\o'
  df = pd.read_csv('Data\overlevelses_ssh.csv')


In [2]:
df = pd.read_csv('Data/formue_cohort.csv')
df = df[(df['KOEN']==1) & (df['ALDER']<100)]

In [3]:
s_data = np.array(df['FGCX'])
a_data = np.array(df['nettoform_u_pf'])

In [4]:
def obj_func(theta,theta_names,mom_data,model,do_print=False):
    
    # printing
    if do_print: 
        print_str = ''
        for i,name in enumerate(theta_names):
            print_str += f'{name}={theta[i]:2.3f} '
        print(print_str)
            
    # calculate moments from simulation
    mom_sim = simulate_moments(theta,theta_names,model,do_print=False)
    
    # equal weighting
    obj = np.sum((mom_data - mom_sim)**2)
    if do_print: 
        print("error", obj)
    
    # return objective and moments for plotting
    return obj

def simulate_moments(theta,theta_names,model,do_print=False):
        
    # update parameters
    for i,name in enumerate(theta_names):
        setattr(model.par,name,theta[i])
    
    # solve and simulate model with these parameters
    model.solve()
    model.simulate()
    
    # return moments from simulated data
    return moment_func(model.sim)

def moment_func(input_data):
    return np.mean(input_data.a,axis=0) # age profile of wealth

In [5]:
theta_names = ("beta", "sigma", "gamma", "mu", "work_cost")

In [6]:
# call numerical optimizer
objective = lambda theta: obj_func(theta,theta_names,a_data,model,do_print=True)

theta_init = np.array([par.beta,par.sigma, par.gamma, par.mu, par.work_cost]) 

res = minimize(objective,theta_init,method='nelder-mead',bounds=[(0.,1.), (0.1,6.), (0.1,6.0), (0.0,5.0),(0,1_000_000)], tol=1, options= {"maxiter":2000})
res

beta=0.980 sigma=1.100 gamma=1.100 mu=0.200 work_cost=1.000 
error 58570102556949.03
beta=0.971 sigma=1.100 gamma=1.100 mu=0.200 work_cost=1.000 
error 66638572669768.945
beta=0.980 sigma=1.155 gamma=1.100 mu=0.200 work_cost=1.000 
error 66733337331295.93
beta=0.980 sigma=1.100 gamma=1.155 mu=0.200 work_cost=1.000 
error 58196135965522.88
beta=0.980 sigma=1.100 gamma=1.100 mu=0.210 work_cost=1.000 
error 58226079458762.945
beta=0.980 sigma=1.100 gamma=1.100 mu=0.200 work_cost=1.050 
error 59478801841028.086
beta=0.976 sigma=1.045 gamma=1.122 mu=0.204 work_cost=1.020 
error 53359111560696.125
beta=0.975 sigma=0.990 gamma=1.133 mu=0.206 work_cost=1.030 
error 53678989029829.85
beta=0.988 sigma=1.078 gamma=1.131 mu=0.206 work_cost=1.028 


Traceback (most recent call last):
  File "f:\MPC\Studenter\Speciale\Peter og Mathias\Code Peter\Design_retirement\model.py", line 201, in simulate
    sim.w[i,t] = wage(par, sol, sim.k[i,t], t)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
f:\MPC\Studenter\Speciale\Peter og Mathias\Code Peter\Design_retirement\model.py:69: SyntaxWarning: invalid escape sequence '\o'
  df = pd.read_csv('Data\overlevelses_ssh.csv')


KeyboardInterrupt: 